In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [3]:
# Chrome webdriver setup
browser = Browser('chrome', 'chromedriver.exe', headless=False)

In [3]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
# Scrape the NASA Mars News page
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

latest = soup.find('div', class_='image_and_description_container')

for article in latest:
    title = latest.find('h3').text
    text = latest.find('div', class_='article_teaser_body').text

In [5]:
print(title)
print(text)

NASA Readies Perseverance Mars Rover's Earthly Twin 
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.


In [6]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [7]:
# Scrape the NASA Jet Propulation Lab website to get the latest featured image
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Click on the links to navigate to the desired page
try:
    browser.links.find_by_partial_text('FULL IMAGE').click()
    try:
        browser.links.find_by_partial_text('more info').click()
    except:
        print('Button 2 Not Found')
except:
    print('Button Not Found')

In [8]:
# Find the featured image url from the soup
image = soup.find('a', class_='fancybox')["data-fancybox-href"]
featured_image_url = "https://www.jpl.nasa.gov" + image

In [9]:
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA01384_ip.jpg


In [10]:
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [11]:
# Scrape the Mars page of the Space Facts website
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [12]:
# Find the facts from the soup
results = soup.find("section", id="secondary")

In [13]:
# Read the data into a Pandas table
tables = pd.read_html(url)

In [14]:
# Put Mars facts into its own table
mars_df = tables[0]
mars_df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [15]:
# Transform the table into a dataframe
mars_fact_table = mars_df.to_html('mars_facts.html', header=False, index=False)

In [16]:
# Put the Mars / Earth comparison data into its own table
mars_earth_df = tables[1]
mars_earth_df.head()

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days


In [17]:
# Transform the table into a dataframe
mars_earth_table = mars_earth_df.to_html('mars_earth_facts.html', index=False)

In [4]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [5]:
# Scrape the website for high res images of each hemisphere
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [20]:
# Get a list of the links to images of each hemisphere
hemisphere_link_list = []

for link in soup.find_all('a', class_="itemLink product-item"):
    if (link.get('href')) not in hemisphere_link_list:
        hemisphere_link_list.append(link.get('href'))

In [22]:
# concatenate the hemisphere link list to the url root to create full urls
url_list = []

for i in range(len(hemisphere_link_list)):
    url_list.append("https://astrogeology.usgs.gov" + hemisphere_link_list[i])

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [23]:
# Visit each url in the url_list to get a link to the high quality image of each hemisphere and create a list
# to hold the links and the hemisphere name
hemisphere_image_urls = []

for url in url_list:
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    hemisphere_image_urls.append({'title': soup.find('h2', class_="title").text[:-9],
                                 'img_url': soup.find(text='Sample').parent['href']})

In [66]:
soup.find(text='Sample').parent['href']

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'